## Install packages

In [ ]:
!poetry install --no-root

## OPENAI settings

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = <openai-api-key>

## LLM

In [65]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

name = llm.predict("I want to open a restaurant for Hungarian food. Suggest a fancy name for this.")
print(name)



"Paprika Palace"


## Prompt template

In [52]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="Hungarian")
print(p)

I want to open a restaurant for Hungarian food. Suggest a fancy name for this.


## Chains

In [53]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Hungarian")

'\n\n"Magyar Delicacies" '

In [54]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Hungarian")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Hungarian food. Suggest a fancy name for this.

> Finished chain.


'\n\n"Goulash Palace"'

### Simple sequental chain

- Restaurant and menu items chanins

In [55]:
# LLM
llm = OpenAI(temperature=0.6)

# Prompt template for restaurant with parameter
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. You are a restaurant advisor, so please suggest a fancy name for it."
)

# Chain - Restaurant name
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

# Prompt template for menu
prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = """
    You are Heston Blumenthal, the famous chef. Blumenthal is regarded as a pioneer of multi-sensory cooking, food pairing and flavour encapsulation. 
    He came to public attention with unusual recipes, such as bacon-and-egg ice cream and snail porridge. 
    His recipes for triple-cooked chips and soft-centred Scotch eggs have been widely imitated. 
    He has advocated a scientific approach to cooking, for which he has been awarded honorary degrees from the universities of Reading, Bristol and London and made an honorary fellow of the Royal Society of Chemistry.
    
    Please suggest some menu items for {restaurant_name}
    """
)

# Chain - menu
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)



- Simple sequental chain

In [56]:
from langchain.chains import SimpleSequentialChain

# Use chains we defined above
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

# Run this chain sequence
content = chain.run("Hungarian")
print(content)


1. "Goulash Explosion" - A deconstructed version of traditional Hungarian goulash, with tender beef, paprika foam, and a crispy potato lattice.

2. "Foie Gras and Tokaji Jelly" - A modern take on the classic Hungarian dish, combining rich foie gras and sweet Tokaji wine jelly.

3. "Duck Confit with Cherry Palinka Reduction" - Slow-cooked duck leg served with a tangy cherry palinka reduction and crispy potato rosti.

4. "Crispy Pork Belly with Spiced Plum Compote" - Crispy pork belly served with a sweet and tangy spiced plum compote and a side of pickled cabbage.

5. "Mushroom and Paprika Risotto" - A creamy risotto made with wild mushrooms and smoked paprika, topped with crispy fried onions.

6. "Beef Stroganoff with Spaetzle" - Tender beef strips cooked in a rich and creamy sauce served over homemade spaetzle noodles.

7. "Langos with Garlic Sour Cream" - A popular Hungarian street food, crispy fried dough topped with a garlic-infused sour cream and your choice of toppings such as ch

### Sequental Chain

- Restaurant name

In [57]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. You are a restaurant advisor, so please suggest a fancy name for it."
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")



- Menu items

In [58]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="menu_items")

    

- Sequental chain

In [59]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

- Run chains

In [60]:
chain({"cuisine": "Hungarian"})

{'cuisine': 'Hungarian',
 'restaurant_name': '\n\n"Paprika Palace"',
 'menu_items': '\n\n1. Paprika chicken skewers with tzatziki dip\n2. Spicy paprika shrimp and chorizo paella\n3. Paprika roasted vegetable and quinoa salad\n4. Paprika crusted salmon with lemon dill sauce\n5. Paprika and garlic roasted potatoes\n6. Paprika marinated pork tenderloin with apple chutney\n7. Paprika spiced beef and vegetable stir-fry\n8. Paprika and feta stuffed mushrooms\n9. Paprika and herb crusted rack of lamb\n10. Paprika sweet potato fries with chipotle mayo.'}

In [ ]:
## Agents

In [72]:
# make sure yoy have installed this package: pip install google-search-results
# from secret_key import serpapi_key
# os.environ['SERPAPI_API_KEY'] = serpapi_key

os.environ['SERPAPI_API_KEY'] = "add your serpapi key here"

In [71]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("When was Elon Musk born? What is his age right now in 2024?")



> Entering new AgentExecutor chain...
 I need to use wikipedia to find Elon Musk's birthdate.
Action: wikipedia
Action Input: "Elon Musk"
Observation: Page: Elon Musk
Summary: Elon Reeve Musk  (; born June 28, 1971) is a businessman and investor known for his key roles in space company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp., the company that operates the social media platform X (formerly known as Twitter), and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI. He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be US$221 billion.
Musk was born in Pretoria to model Maye and businessman and engineer Errol Musk, and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk later transferred to the Univer

'Elon Musk will be 53 years old in 2024.'

In [81]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["serpapi", "llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("What was the GDP of US in 2022, plus 5?")



> Entering new AgentExecutor chain...
 I should use the calculator to add 5 to the GDP of US in 2022.
Action: Calculator
Action Input: 5 and GDP of US in 2022
Observation: Answer: 22547800000005
Thought: Now I know the final answer.
Final Answer: 22547800000005

> Finished chain.


'22547800000005'

## Memory